In [ ]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

# # Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

In [5]:
#I create the bus 

n.madd("Bus", ["onebus"], x=config["microgrids_list"]["micA"]["lon"], y=config["microgrids_list"]["micA"]["lat"], carrier="AC", v_nom=20)



Index(['onebus'], dtype='object')

In [3]:
#TODO file cost last two lines are completely invended
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [4]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [5]:
def _add_missing_carriers_from_costs(n, costs, carriers):
    missing_carriers = pd.Index(carriers).difference(n.carriers.index)
    if missing_carriers.empty:
        return

    emissions_cols = (
        costs.columns.to_series().loc[lambda s: s.str.endswith("_emissions")].values
    )
    suptechs = missing_carriers.str.split("-").str[0]
    emissions = costs.loc[suptechs, emissions_cols].fillna(0.0)
    emissions.index = missing_carriers

In [6]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [33]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )
    
    max_hours = elec_config["max_hours"]           
    costs.loc["battery"] = costs_for_storage(                      
            costs.loc["lithium"],                            #line 119 in file costs.csv' which was battery storage was modified into lithium (same values left)
            costs.loc["battery inverter"],
            max_hours=max_hours["battery"],
    )
    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
            costs.loc["lead acid"],                          #line 120 in file 'costs.csv' which was battery storage was modified into lithium (same values left)
            costs.loc["battery inverter"],
            max_hours=max_hours["battery"],
    )
    
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [34]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [9]:
def attach_wind_and_solar(n, costs, tech_modelling, extendable_carriers):

    _add_missing_carriers_from_costs(n, costs, tech_modelling)

    for tech in tech_modelling:
       
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:
            
            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]
          
            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus=["onebus"],
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(), #guardare il config 
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_set=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas().reindex(n.snapshots),
            )


In [10]:
attach_wind_and_solar(
    n,
    costs,
    config["tech_modelling"]["general_vre"],
    config["electricity"]["extendable_carriers"],
    )

In [11]:
carrier_dict = {
        "ocgt": "OCGT",
        "ccgt": "CCGT",
        "bioenergy": "biomass",
        "ccgt, thermal": "CCGT",
        "hard coal": "coal",
    }

ppl_fn=r'C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//powerplants.csv'
ppl=pd.read_csv(ppl_fn, index_col=0, dtype={"bus":"str"}).rename(columns=str.lower).drop(columns=["efficiency"]).replace({"carrier": carrier_dict})

In [12]:

# def attach_conventional_generators(n, costs, ppl, tech_modelling, conventional_carriers): 
    
#     _add_missing_carriers_from_costs(n, costs, tech_modelling)
#     # carriers = set(conventional_carriers) | set(extendable_carriers["Generator"])

#     # ppl = (
#     #     ppl.query("carrier in @carriers")
#     #     .join(costs, on="carrier", rsuffix="_r")
#     #     .rename(index=lambda s: "C" + str(s))
#     # )
#     # ppl["efficiency"] = ppl.efficiency.fillna(ppl.efficiency)

#     for tech in tech_modelling:

#         n.madd(
#         "Generator", 
#         ppl.index,
#         " " + tech,
#         bus=["onebus"],
        
#     )

#         p_nom_min=ppl.p_nom.where(ppl.carrier.isin(conventional_carriers), 0),
#         p_nom=ppl.p_nom.where(ppl.carrier.isin(conventional_carriers), 0),
#         p_nom_extendable=ppl.carrier.isin(extendable_carriers["Generator"]),
#         efficiency=ppl.efficiency,
#         marginal_cost=ppl.marginal_cost,
#         capital_cost=ppl.capital_cost,
#         build_year=ppl.datein.fillna(0).astype(int),
#         lifetime=(ppl.dateout - ppl.datein).fillna(np.inf),
    

In [13]:
# attach_conventional_generators(n, costs, ppl, config["tech_modelling"]["conv_techs"], config["electricity"]["conventional_carriers"])

In [ ]:
costs

In [15]:
def attach_storageunits(n, costs,technologies, extendable_carriers ):

    buses_i = n.buses.index

    for tech in technologies:
        
        n.madd(
            "StorageUnit",
            buses_i, 
            " " + tech, 
            bus=["onebus"],
            carrier=tech,
            p_nom_extendable=True,
            capital_cost=costs.at[tech, "capital_cost"],
            marginal_cost=costs.at[tech, "marginal_cost"],
            # efficiency_store=costs.at[lookup_store[tech], "efficiency"],
            # efficiency_dispatch=costs.at[lookup_dispatch[tech], "efficiency"],
            # max_hours=max_hours[tech],
            cyclic_state_of_charge=True
            
        )

In [16]:
attach_storageunits(n, 
                    costs,
                    config["tech_modelling"]["storage_techs"],
                    config["electricity"]["extendable_carriers"],
                    )

In [17]:
n.storage_units

,bus,carrier,p_nom_extendable,capital_cost,marginal_cost,cyclic_state_of_charge,control,type,p_nom,p_nom_min,...,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
onebus lithium,onebus,lithium,True,12409.436462,0.0,True,PQ,,0.0,0.0,...,0.0,False,NaN,True,1.0,1.0,1.0,0.0,0.0,0.0
onebus lead acid,onebus,lead acid,True,0.000000,0.0,True,PQ,,0.0,0.0,...,0.0,False,NaN,True,1.0,1.0,1.0,0.0,0.0,0.0


In [10]:

#I create the load file
n_load=35 #number of loads
import numpy as np
p_set=np.random.rand(len(n.snapshots),(n_load))
array=p_set*100

In [11]:
import pandas as pd
df=pd.DataFrame(array)

filepath = 'my_file.xlsx'

df.to_excel(filepath, index=False)



In [12]:
load_df=pd.read_excel(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx')


In [13]:

load_df=load_df.set_index([n.snapshots])
load_df.index.names=['time']
print(load_df)

                            0          1          2          3          4   \
time                                                                         
2013-01-01 00:00:00  89.268586  96.427755  10.889164  55.252615  48.224946   
2013-01-01 01:00:00   8.951014  47.191324  32.045252  35.249176  94.124216   
2013-01-01 02:00:00  85.965574  48.261010  66.888110  74.581181  99.357900   
2013-01-01 03:00:00  50.905691  70.408773  66.203870   1.136231  70.998455   
2013-01-01 04:00:00  46.505674  58.082270  53.067450  38.732952  70.986373   
...                        ...        ...        ...        ...        ...   
2013-12-31 19:00:00  57.426152  69.736037  21.206181   8.545961  19.860242   
2013-12-31 20:00:00  54.718517  79.780202  14.974637  44.350101  16.104340   
2013-12-31 21:00:00  51.323545  31.733624  89.469989  24.919291  98.955221   
2013-12-31 22:00:00  53.291795  98.530025  59.871263   8.508648  23.415766   
2013-12-31 23:00:00  25.145650  58.122272   9.265614  96.995512 

In [14]:
index=pd.Index( list(range(n_load)))
type(index)

pandas.core.indexes.numeric.Int64Index

In [6]:
def attach_load(n, load_paths, index, load_df, tech_modelling):
    
    n.madd("Load", index ,bus=["onebus"], carrier="AC", p_set=load_df)




In [7]:
load_paths=r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\my_file.xlsx'

In [16]:
attach_load(n, load_paths, index, load_df, config["tech_modelling"]["load_carriers"])

In [27]:
print(n)

PyPSA Network
Components:
 - Bus: 1
 - Generator: 105
 - Load: 35
 - StorageUnit: 2
Snapshots: 8760


In [28]:
# Optimization
from pypsa.linopf import ilopf

solver_name="gurobi"

n.lopf(n.snapshots, solver_name=solver_name, pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 8.06s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-07
Read LP format model from file C:\Users\denis\AppData\Local\Temp\pypsa-problem-faqxks1g.lp
Reading time = 4.84 seconds
obj: 1971000 rows, 972468 columns, 3032023 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1971000 rows, 972468 columns and 3032023 nonzeros
Model fingerprint: 0x0255ff06
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e-02, 1e+05]
  Bounds range     [4e+00, 2e+05]
  RHS range        [1e+03, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1917377 rows and 927499 columns
Presolve time: 2.04s
Presolved: 53623 rows, 44969 columns, 152109 nonzeros

Ordering time: 0.08s

Barrier statistics:
 Dense cols : 49
 AA' NZ     : 2.637e+05
 Factor NZ  : 9.286e+05 (roughly 50 MB of memory)
 Factor

INFO:pypsa.linopf:Optimization successful. Objective value: 2.92e+10


('ok', 'optimal')